In [1]:
!pip install pyspark
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=ca034bfe6462b4b480dbf50eae2314cfe90e24ef9e9472cad2c4fe30acd6e8be
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics

In [3]:
spark = SparkSession.builder.appName("IncomeClassifierModel").getOrCreate()

In [4]:
spark

In [5]:
from google.colab import files
uploaded = files.upload()

Saving income(1).csv to income(1).csv


In [6]:
data_path = list(uploaded.keys())[0]
data = spark.read.csv(data_path, header=True, inferSchema=True)

In [7]:
data = data.dropna()


In [8]:
data.printSchema()
data.show(5)

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- weight: double (nullable = true)
 |-- education: string (nullable = true)
 |-- education_years: double (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital_gain: double (nullable = true)
 |-- capital_loss: double (nullable = true)
 |-- hours_per_week: double (nullable = true)
 |-- citizenship: string (nullable = true)
 |-- income_class: string (nullable = true)

+---+-----------------+--------+----------+---------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------------+
|age|        workclass|  weight| education|education_years|     marital_status|        occupation|  relationship|  race|    sex|capital_gain|capital_loss|hours_per_week|

In [9]:
categorical_columns = [col for col in data.columns if data.schema[col].dataType == 'string' and col != 'income_class']
indexers = [StringIndexer(inputCol=col, outputCol=col + "_index").fit(data) for col in categorical_columns]

# Apply StringIndexer transformations
for indexer in indexers:
    data = indexer.transform(data)

# Inspect the schema and data to verify indexed columns
data.printSchema()
data.show(5)


root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- weight: double (nullable = true)
 |-- education: string (nullable = true)
 |-- education_years: double (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital_gain: double (nullable = true)
 |-- capital_loss: double (nullable = true)
 |-- hours_per_week: double (nullable = true)
 |-- citizenship: string (nullable = true)
 |-- income_class: string (nullable = true)

+---+-----------------+--------+----------+---------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------------+
|age|        workclass|  weight| education|education_years|     marital_status|        occupation|  relationship|  race|    sex|capital_gain|capital_loss|hours_per_week|

In [18]:
for column in categorical_columns:
    indexer = StringIndexer(inputCol=column, outputCol=column + "_index")
    data = indexer.fit(data).transform(data)

IllegalArgumentException: requirement failed: Output column workclass_index already exists.

In [13]:
from pyspark.sql.types import StringType

In [14]:
categorical_columns = [c for c, t in data.dtypes if t == 'string' and c != 'income_class']
print("Categorical columns:", categorical_columns)

Categorical columns: ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'citizenship']


In [19]:
data.printSchema()
data.show(5)

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- weight: double (nullable = true)
 |-- education: string (nullable = true)
 |-- education_years: double (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital_gain: double (nullable = true)
 |-- capital_loss: double (nullable = true)
 |-- hours_per_week: double (nullable = true)
 |-- citizenship: string (nullable = true)
 |-- income_class: string (nullable = true)
 |-- workclass_index: double (nullable = false)
 |-- education_index: double (nullable = false)
 |-- marital_status_index: double (nullable = false)
 |-- occupation_index: double (nullable = false)
 |-- relationship_index: double (nullable = false)
 |-- race_index: double (nullable = false)
 |-- sex_index: double (nullable = false)
 |-- citizenship_index: double (nu

In [20]:
numerical_columns = [c for c, t in data.dtypes if t in ['int', 'double'] and c != 'income_class']
print("Numerical columns:", numerical_columns)

Numerical columns: ['age', 'weight', 'education_years', 'capital_gain', 'capital_loss', 'hours_per_week', 'workclass_index', 'education_index', 'marital_status_index', 'occupation_index', 'relationship_index', 'race_index', 'sex_index', 'citizenship_index']


In [21]:
assembler_numerical = VectorAssembler(inputCols=numerical_columns, outputCol="numerical_features")
data = assembler_numerical.transform(data)

In [22]:
data.select("numerical_features").show(5, truncate=False)

+-------------------------------------------------------------------+
|numerical_features                                                 |
+-------------------------------------------------------------------+
|[39.0,77516.0,13.0,2174.0,0.0,40.0,4.0,2.0,1.0,3.0,1.0,0.0,0.0,0.0]|
|(14,[0,1,2,5,6,7,9],[50.0,83311.0,13.0,13.0,1.0,2.0,2.0])          |
|(14,[0,1,2,5,8,9,10],[38.0,215646.0,9.0,40.0,2.0,9.0,1.0])         |
|(14,[0,1,2,5,7,9,11],[53.0,234721.0,7.0,40.0,5.0,9.0,1.0])         |
|[28.0,338409.0,13.0,0.0,0.0,40.0,0.0,2.0,0.0,0.0,4.0,1.0,1.0,9.0]  |
+-------------------------------------------------------------------+
only showing top 5 rows



In [23]:
scaler = StandardScaler(inputCol="numerical_features", outputCol="scaled_numerical_features", withMean=True, withStd=True)
scaler_model = scaler.fit(data)
data = scaler_model.transform(data)

In [24]:
data.select("scaled_numerical_features").show(5, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|scaled_numerical_features                                                                                                                                                                                                                                                                    |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[0.03067008637999638,-1.0635944124434717,1.1347213388479223,0.14845061558793698,-0.2166562000280365,-0.035428902921321385,1.91347581007

In [25]:
feature_columns = ["scaled_numerical_features"] + [col + "_index" for col in categorical_columns]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data = assembler.transform(data)

In [26]:
data.select("features").show(5, truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                                                                                                                                                     |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[0.03067008637999638,-1.063594412443471

In [27]:
label_indexer = StringIndexer(inputCol="income_class", outputCol="label").fit(data)
data = label_indexer.transform(data)

In [28]:
data.select("label").show(5)

+-----+
|label|
+-----+
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
+-----+
only showing top 5 rows



In [29]:
data = data.select("features", "label")

In [30]:
data.show(5, truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|features                                                                                                                                                                                                                                                                                                                     |label|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|[0.03067008637999638,

In [31]:
train_data, test_data = data.randomSplit([0.7, 0.3], seed=42)

In [33]:
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxBins=50)
dt_model = dt.fit(train_data)

In [34]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features", maxBins=50)
rf_model = rf.fit(train_data)

In [35]:
dt_predictions = dt_model.transform(test_data)
rf_predictions = rf_model.transform(test_data)

In [43]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
dt_accuracy = evaluator.evaluate(dt_predictions)
print(f"Decision Tree Accuracy: {dt_accuracy:.2f}")

AssertionError: 

In [37]:
rf_accuracy = evaluator.evaluate(rf_predictions)
print(f"Random Forest Accuracy: {rf_accuracy:.2f}")

Random Forest Accuracy: 0.86


In [38]:
def print_confusion_matrix(predictions, model_name):
    prediction_and_labels = predictions.select("prediction", "label").rdd
    metrics = MulticlassMetrics(prediction_and_labels)
    confusion_matrix = metrics.confusionMatrix().toArray()
    print(f"Confusion Matrix for {model_name}:\n{confusion_matrix}")

In [39]:
print_confusion_matrix(dt_predictions, "Decision Tree")

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Confusion Matrix for Decision Tree:
[[7015.  303.]
 [1199. 1169.]]


In [40]:
print_confusion_matrix(rf_predictions, "Random Forest")

Confusion Matrix for Random Forest:
[[7053.  265.]
 [1104. 1264.]]


In [41]:
if rf_accuracy > dt_accuracy:
    print("Random Forest is the better model.")
else:
    print("Decision Tree is the better model.")


Random Forest is the better model.


In [42]:
spark.stop()